### Word Group In Youtube

In [1]:
import os
import multiprocessing
#import multiprocessing as mp
from multiprocessing import Process, Manager, Pool, Queue
from itertools import islice
from collections import Counter
import re
import pandas as pd
import numpy as np
import glob
import nltk
from nltk import word_tokenize
from functools import reduce
from pathlib import Path
import shutil

In [2]:
nprocs = multiprocessing.cpu_count()
print(f"Number of CPU cores: {nprocs}")

Number of CPU cores: 16


In [3]:
# language pair
lang_folder = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> target language for learner
#lang_pair = "English"  # Arabic, English, French, German, Turkish, Spanish, Portuguese, Dutch, Italian ==> native language

# adding native word to shared word
word_start = 0  # 0  # native word start index
word_end = 1000  # native word end index 28,200,1000,5000,10000, 20000, 40000

# word ratio
if word_end == 28 or word_end == 200:
    unique_word_ratio = 70
    include_wordgroup_ratio = 70
elif word_end == 1000 or word_end == 5000:
    unique_word_ratio = 80
    include_wordgroup_ratio = 80
elif word_end == 10000:
    unique_word_ratio = 85
    include_wordgroup_ratio = 85
else:
    unique_word_ratio = 90
    include_wordgroup_ratio = 90

# word usage
if word_end == 28 or word_end == 200 or word_end == 1000 or word_end == 5000 or word_end == 10000:
    word_usage_min = 10
    word_usage_max = 100
elif word_end == 20000:
    word_usage_min = 7
    word_usage_max = 100
else:
    word_usage_min = 5
    word_usage_max = 100

# use for manuel set
#unique_word_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 ; 10000 = 85 ; 20000,40000 = 90
#include_wordgroup_ratio = 70  # 28,200 = 70 ; 1000,5000 = 80 : 10000 = 85 ; 20000,40000 = 90
#word_usage_min = 10  # 28,200,1000,5000,10000 = 10 ; 20000 = 7 ; 40000 = 5
#word_usage_max = 100


# sentence ratio and time shift
adjust_sent_word_ratio = 15  #28,200 = 15 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
#adjust_text_word_ratio = 5  #28,200 = 5 ; 1000,5000,10000 = 25 ; 20000 = 35 ; 40000 = 45
shift = 0.3  # sentence time shift

# func select
if word_end == 28:
    sentence_word_count_number = 2
else:
    sentence_word_count_number = 5   
word_group_adjust_select = False  # True, False; True for selecting word group according to sentence_word_count_number in video_id text analysis
                                  # False for selecting word group according to max word group length 

# shared word frequency
word_frequency = True  # True, False

# prefix suffix file
prefix_suffix = False  # True, False  # True for adding prefix suffix word
native_word = True # True for adding native word

# adding output file extention
if prefix_suffix & native_word:
    file_ext = "5"
elif (not prefix_suffix) & native_word:
    file_ext = "6"
else:
    file_ext = "7"              

# 5 => for only native word with prefix suffix.
# 6 => for only native word without prefix suffix.

print(f"word_end:{word_end}\nfile_ext:{file_ext}\nunique_word_ratio:{unique_word_ratio}\n\
include_wordgroup_ratio:{include_wordgroup_ratio}\nword_usage_min:{word_usage_min}\nword_usage_max:{word_usage_max}")

word_end:20000
file_ext:6
unique_word_ratio:90
include_wordgroup_ratio:90
word_usage_min:7
word_usage_max:100


In [4]:
path = f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.capitalize()}/\
Language Level/Data/1-Word Group In Youtube Sentence"

Path(path).mkdir(parents=True, exist_ok=True)

In [5]:
def word_usage_result(word_list, df_target, target_column, target_opt_column, word_usage_min, word_usage_max):
    '''
    word_usage_result(word_list, df_ngram_pair, "threegram", "frequency", 1, 5) \n
    word_list is a list, df_target is a dateframe, target_column is df_target dataframe target column, \n
    target_opt_column is df_target dataframe opt_target column, \n
    word_usage_min and word_usage_max word usage condition.
    '''    
    word_num_dict = {}
    for i in word_list:
        word_num_dict[f"{i}"] = 0
    
    result_list_select = []
    var_list = []
    for i in range(len(df_target)):
        target_value = df_target.loc[i,f"{target_column}"]
        opt_value = df_target.loc[i,f"{target_opt_column}"]
        words = word_tokenize(target_value)   
        temp_list = [word for word in words]
        temp_list = temp_list + var_list
        # word count for max
        dict_list_count = Counter(temp_list)
        count_list = list(dict_list_count.values())
        # word count for min
        count_list2 = list(word_num_dict.values())
    
        if any([True if i>word_usage_max else False for i in count_list]) or not(any([True if j<word_usage_min else False for j in count_list2])):
            pass
        else:
            var_list = temp_list
            result_list_select.append([target_value,opt_value]) 
    
            for item2 in dict_list_count.items(): 
                word_num_dict[item2[0]] = item2[1]        
    df_result = pd.DataFrame(result_list_select, columns=[f"{target_column}",f"{target_opt_column}"])
    df_result.sort_values(by=f"{target_opt_column}", ascending=False, inplace=True)
    df_result.reset_index(drop=True, inplace=True)
    
    return df_result

In [6]:
def exract_word_group(text, word_list):
    '''
    exract_word_group(text, word_list): \n
    text is a string sentence, word_list occurs words 
    '''
    words = re.findall(r"\w+", text, re.UNICODE)
    index_list = []

    for i in range(len(words)):
        if words[i] in word_list:
            index_list.append(i)
        else:
            pass

    all_index_list = []
    var_index_list = []
    for j in range(len(index_list)):
        try:
            var1 = index_list[j] + 1  
            var2 = index_list[j+1]
        except:
            var1 = index_list[j] + 1  
            var2 = index_list[-1]
        if var1 == var2:
            var3 = index_list[j]
            var_index_list.append(var3)
        else:
            var3 = index_list[j]
            var_index_list.append(var3)
            var4 = list(var_index_list)
            all_index_list.append(var4)
            var_index_list = []

    text_list = []
    for k in max(all_index_list, key=len):  # any error convert k to i
        word = words[k]
        text_list.append(word)
        if len(text_list) >= sentence_word_count_number:
            text = " ".join(text_list)
        else:
            text = f"sentencte_word_count_less_than_{sentence_word_count_number}"
    
    return text 

In [7]:
def df_exract_word_group(df, source_text_column, opt_column, word_list, sent_len=False, sent_len_num=2):
    '''
    df_exract_word_group(df_adjust_text_ratio, sentence, video_id, word_list, sent_len=False, sent_len_num=2): \n
    df_adjust_text_ratio is a dataframe and it includes sentence and video_id columns. \n
    sentence is a string sentence. video_id is optional column value. word_list occurs words \n
    sent_len is sentence length condition. sent_len_num is sentence occurs how many word at least. 
    '''
    df.reset_index(drop=True, inplace=True)
    df_new = pd.DataFrame()
    for i in range(len(df)):
        source_text = df.loc[i,f"{source_text_column}"]
        opt_var = df.loc[i,f"{opt_column}"]
        words = re.findall(r"\w+", source_text, re.UNICODE)
        
        index_list = []
        for j in range(len(words)):
            if words[j] in word_list:
                index_list.append(j)
            else:
                pass

        all_index_list = []
        var_index_list = []
        for k in range(len(index_list)):
            try:
                var1 = index_list[k] + 1  
                var2 = index_list[k+1]
            except:
                var1 = index_list[k] + 1  
                var2 = index_list[-1]
            if var1 == var2:
                var3 = index_list[k]
                var_index_list.append(var3)
            else:
                var3 = index_list[k]
                var_index_list.append(var3)
                var4 = list(var_index_list)
                all_index_list.append(var4)
                var_index_list = []

        text_all_list = []
        for m in all_index_list:
            text_list = [] 
            for n in m:
                word = words[n]
                text_list.append(word)
                if sent_len:
                    if len(text_list) >= sent_len_num:
                        text = " ".join(text_list)
                        text_all_list.append(text)
                    else:
                        pass
                else:
                   text = " ".join(text_list)
                   text_all_list.append(text) 
            
        for search_string in text_all_list:
            #df_var["index"] = i
            #df_var["search_string"] = search_string
            #df_var["sentence"] = source_text
            #df_var["video_id"] = opt_var
            df_list = []
            df_list.append([i,search_string,source_text,opt_var])
            df_var = pd.DataFrame(df_list,columns=["index","search_string",f"{source_text_column}",f"{opt_column}"])
            df_new = pd.concat([df_new, df_var], axis=0)

    return df_new 

In [8]:
def word_group_time_loc(df, search, start_sent, end_sent, sent, sent_video_id):
    '''
    word_group_time_loc(df_search_result, "search_string", "start_time", "end_time", "sentence", "video_id")
    '''
    df.reset_index(drop=True, inplace=True)  # will test
    word_time_loc_list = []
    for i in range(len(df)):
        word = df.loc[i,f"{search}"]
        start_time = df.loc[i,f"{start_sent}"]
        end_time = df.loc[i,f"{end_sent}"]
        sentence = df.loc[i,f"{sent}"]
        video_id = df.loc[i,f"{sent_video_id}"]
        time_length = end_time-start_time
        sentence_length = len(sentence)
        time_length_ratio = time_length/sentence_length
        loc_list = []
        for j in re.finditer(fr"(?:\s|^){word}(?:\s|$)", sentence, re.IGNORECASE|re.UNICODE):
            loc_list.append(j)
            start = loc_list[0].start()
            end = loc_list[0].end()
            start_loc = start_time+(start*time_length_ratio)
            end_loc = start_time+(end*time_length_ratio)
        word_time_loc_list.append([word,start_loc,end_loc,sentence,video_id])
    df_word_time_loc = pd.DataFrame(word_time_loc_list, columns=[f"{search}",f"{start_sent}",f"{end_sent}",f"{sent}",f"{sent_video_id}"])

    return df_word_time_loc

In [9]:
def repetation_ratio(text):
    '''
    repetation_ratio(text) 
    text is a like as sentence and def compute repetation word ratio\n
    repetation_ratio("yok bir şey yok bir şey yok bir şey") 
    '''
    word_list = word_tokenize(text)
    word_num = len(word_list)
    word_unique_num = len(set(word_list))
    ratio = int((word_unique_num/word_num)*100)
    
    return ratio

In [10]:
df_word_all = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/{lang_folder.lower().capitalize()}/Deployment/Data/Word/Word_Merge_Preprocess.xlsx")
df_word_all

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
552149,fruitcocktail,6
552150,andthesunlightshining,6
552151,upravo,6
552152,yagawa,6


In [11]:
df_word_select = df_word_all.iloc[word_start:word_end,]

In [12]:
# Option
if prefix_suffix:
    df_word = pd.read_excel(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Web Scrapping/Data/{lang_folder.capitalize()}/{lang_folder.capitalize()}/{lang_folder.capitalize()}_{word_end}_Word_Prefix_Suffix_Custom_Result_Manuel.xlsx")
    df_word = df_word.loc[:,["word","frequency"]]
    df_word = pd.concat([df_word,df_word_select], axis=0)
    df_word.drop_duplicates(inplace=True)    
    df_word.sort_values(by="frequency", ascending=False, inplace=True)
    df_word.reset_index(drop=True, inplace=True)
else:
    df_word = df_word_select

if native_word:
    df_word
else:
    df_word = df_word.head(0)

df_word

,word,frequency
0,you,102069964
1,i,94447074
2,the,77481215
3,to,58281119
4,is,50852895
...,...,...
19995,mais,2711
19996,captions,2711
19997,dysfunctional,2711
19998,revere,2711


In [13]:
disable_list = ["sex","seks","seksi","sexy","sexe","seksüel","sexuell","gey","gay","lezbiyen","lesbienne","eşcinsel","mastürbasyon","masturbation","erotik","érotique", \
"bikini","penis","vagina","vajina","fetish","fetiş","fetishy","erotic","erotik","sexdom","kondom","condom","dildo","fetisj","hétérosexuel","féticher","fétiche","homosexuel"\
"ereksiyon","erectie","erection","érection","homoseksüel","prezervatif","préservatif","ass","fetisch","fetiche"]

In [14]:
word_select = df_word["word"].values.tolist()
word_select_set = set(word_select)
disable_word_set = set(disable_list)

In [15]:
word_list = list(word_select_set.difference(disable_word_set))
len(word_list)

19976

#### Youtube Sentence Analysis

In [16]:
df_youtube_sentence = pd.read_csv(f"/media/kurubal/SSD/Data Scientist/Work/Modern Ways/Project/Youtube/Result/{lang_folder.capitalize()}/Sentence Clean Merge/Clean_Youtube_Sentence_Merge_Result.csv")
df_youtube_sentence

,start_time,end_time,sentence,video_id
0,00:00:29.416,00:00:33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y
1,00:00:33.810,00:00:36.876,shivering cold and snow,tPffr9WPl9Y
2,00:00:37.150,00:00:39.000,what else do you wish for in this festive season,tPffr9WPl9Y
3,00:00:39.257,00:00:43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y
4,00:00:43.796,00:00:50.094,so today we will explore different places in e...,tPffr9WPl9Y
...,...,...,...,...
5742563,00:02:59.400,00:03:03.030,wounded and,78VHQeebbWM
5742564,00:03:03.030,00:03:10.590,general staff of the armed forces of ukraine o...,78VHQeebbWM
5742565,00:03:10.590,00:03:13.159,facebook on december,78VHQeebbWM
5742566,00:03:13.159,00:03:16.769,you can express your thoughts in the comments ...,78VHQeebbWM


In [17]:
df_youtube_sentence['start_time'] = pd.to_timedelta(df_youtube_sentence['start_time']) # data type converted timedelta for second 
df_youtube_sentence['end_time'] = pd.to_timedelta(df_youtube_sentence['end_time'])

In [18]:
df_youtube_sentence['start_time'] = df_youtube_sentence['start_time'].apply(lambda x: x.total_seconds()) # convert seconds
df_youtube_sentence['end_time'] = df_youtube_sentence['end_time'].apply(lambda x: x.total_seconds())
df_youtube_sentence.reset_index(inplace=True)  # adding index column
df_youtube_sentence

,index,start_time,end_time,sentence,video_id
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y
...,...,...,...,...,...
5742563,5742563,179.400,183.030,wounded and,78VHQeebbWM
5742564,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM
5742565,5742565,190.590,193.159,facebook on december,78VHQeebbWM
5742566,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM


In [19]:
#df_youtube_sentence[df_youtube_sentence["index"] == 1287503]

In [20]:
# import os
# from multiprocessing import Process, Manager, Pool, Queue
manager = multiprocessing.Manager()
result_list = manager.list()
word_set = set(word_list)
index_num = list(range(len(df_youtube_sentence)))

def sentence_word_ratio(index_num):
    index = df_youtube_sentence.loc[index_num,"index"]
    start = df_youtube_sentence.loc[index_num,"start_time"]
    end = df_youtube_sentence.loc[index_num,"end_time"]
    sentence = df_youtube_sentence.loc[index_num,"sentence"]
    videoid = df_youtube_sentence.loc[index_num,"video_id"]

    sent_word = re.findall(r"\w+", sentence, re.UNICODE)
    sent_word_set = set(sent_word)
    intersect_word = list(word_set.intersection(sent_word_set))
    different_word = list(sent_word_set.difference(word_set))
    word_ratio = round(((len(intersect_word)/(len(sent_word)+0.001))*100),1)
    different = ", ".join(different_word)
    intersect = ", ".join(intersect_word)

    result_list.append([index,start,end,sentence,videoid,word_ratio,different,intersect])  
    
   
if __name__ == '__main__':
    # with Pool(16) as p:
    with Pool(nprocs-2) as p: # Pool number CPU sayısına eşit olursa tüm CPU lar çalışır
        p.map(sentence_word_ratio, index_num)

In [21]:
df_sentence_ratio_result = pd.DataFrame(list(result_list), columns=["index","start_time","end_time","sentence","video_id","word_ratio","different_word","intersect_word"])
df_sentence_ratio_result.sort_values(by="index", ascending=True, inplace=True)
df_sentence_ratio_result.reset_index(drop=True, inplace=True)
df_sentence_ratio_result

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,62.5,,"bell, all, way, the, jingle"
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,100.0,,"and, snow, shivering, cold"
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,100.0,,"else, festive, do, this, what, wish, in, you, ..."
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,85.7,,"herself, of, colours, city, the, has, and, lig..."
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,100.0,,"being, how, places, different, and, today, so,..."
...,...,...,...,...,...,...,...,...
5742563,5742563,179.400,183.030,wounded and,78VHQeebbWM,100.0,,"and, wounded"
5742564,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM,92.9,,"of, forces, staff, the, information, has, been..."
5742565,5742565,190.590,193.159,facebook on december,78VHQeebbWM,100.0,,"on, facebook, december"
5742566,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM,100.0,,"express, thoughts, the, your, comments, sectio..."


In [22]:
df_sentence_ratio_result["different_word"] = df_sentence_ratio_result["different_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result["intersect_word"] = df_sentence_ratio_result["intersect_word"].apply(lambda x: np.nan if x == "" else x)
df_sentence_ratio_result 

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,62.5,NaN,"bell, all, way, the, jingle"
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,100.0,NaN,"and, snow, shivering, cold"
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,100.0,NaN,"else, festive, do, this, what, wish, in, you, ..."
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,85.7,NaN,"herself, of, colours, city, the, has, and, lig..."
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,100.0,NaN,"being, how, places, different, and, today, so,..."
...,...,...,...,...,...,...,...,...
5742563,5742563,179.400,183.030,wounded and,78VHQeebbWM,100.0,NaN,"and, wounded"
5742564,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM,92.9,NaN,"of, forces, staff, the, information, has, been..."
5742565,5742565,190.590,193.159,facebook on december,78VHQeebbWM,100.0,NaN,"on, facebook, december"
5742566,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM,100.0,NaN,"express, thoughts, the, your, comments, sectio..."


In [23]:
df_adjust_sentence_ratio = df_sentence_ratio_result[df_sentence_ratio_result["word_ratio"] >= adjust_sent_word_ratio]
df_adjust_sentence_ratio.reset_index(inplace=True, drop=True)
df_adjust_sentence_ratio

,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,62.5,NaN,"bell, all, way, the, jingle"
1,1,33.810,36.876,shivering cold and snow,tPffr9WPl9Y,100.0,NaN,"and, snow, shivering, cold"
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,100.0,NaN,"else, festive, do, this, what, wish, in, you, ..."
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,85.7,NaN,"herself, of, colours, city, the, has, and, lig..."
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,100.0,NaN,"being, how, places, different, and, today, so,..."
...,...,...,...,...,...,...,...,...
5722140,5742563,179.400,183.030,wounded and,78VHQeebbWM,100.0,NaN,"and, wounded"
5722141,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM,92.9,NaN,"of, forces, staff, the, information, has, been..."
5722142,5742565,190.590,193.159,facebook on december,78VHQeebbWM,100.0,NaN,"on, facebook, december"
5722143,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM,100.0,NaN,"express, thoughts, the, your, comments, sectio..."


In [24]:
df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))
df_adjust_sentence_ratio = df_adjust_sentence_ratio[~(df_adjust_sentence_ratio["search_string"]==f"sentencte_word_count_less_than_{sentence_word_count_number}")]
#df_adjust_sentence_ratio.reset_index(drop=True, inplace=True)
df_adjust_sentence_ratio

<ipython-input-24-ca15520b3270>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_adjust_sentence_ratio.loc[:,"search_string"] = df_adjust_sentence_ratio.loc[:,"sentence"].map(lambda x: exract_word_group(x, word_list))


,index,start_time,end_time,sentence,video_id,word_ratio,different_word,intersect_word,search_string
0,0,29.416,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y,62.5,NaN,"bell, all, way, the, jingle",jingle bell jingle bell jingle all the way
2,2,37.150,39.000,what else do you wish for in this festive season,tPffr9WPl9Y,100.0,NaN,"else, festive, do, this, what, wish, in, you, ...",what else do you wish for in this festive season
3,3,39.257,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y,85.7,NaN,"herself, of, colours, city, the, has, and, lig...",the city of edinburgh has decorated herself wi...
4,4,43.796,50.094,so today we will explore different places in e...,tPffr9WPl9Y,100.0,NaN,"being, how, places, different, and, today, so,...",so today we will explore different places in e...
5,5,50.504,51.928,so lets go with me,tPffr9WPl9Y,100.0,NaN,"go, me, so, with, lets",so lets go with me
...,...,...,...,...,...,...,...,...,...
5722138,5742561,167.800,173.610,it is reported that as a result of artillery f...,78VHQeebbWM,93.7,NaN,"as, of, russian, fire, damage, it, that, a, re...",it is reported that as a result of artillery f...
5722139,5742562,173.610,179.400,the city of starobilsk luhansk region the russ...,78VHQeebbWM,72.7,"starobilsk, luhansk","of, russian, city, the, lost, region, about, t...",region the russian troops lost about
5722141,5742564,183.030,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM,92.9,NaN,"of, forces, staff, the, information, has, been...",general staff of the armed forces of ukraine o...
5722143,5742566,193.159,196.769,you can express your thoughts in the comments ...,78VHQeebbWM,100.0,NaN,"express, thoughts, the, your, comments, sectio...",you can express your thoughts in the comments ...


In [25]:
df_word_group_time_loc_in_sent_result = word_group_time_loc(df_adjust_sentence_ratio, "search_string", "start_time", "end_time", "sentence", "video_id")
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,jingle bell jingle bell jingle all the way,29.416000,33.605,jingle bell jingle bell jingle all the way,tPffr9WPl9Y
1,what else do you wish for in this festive season,37.150000,39.000,what else do you wish for in this festive season,tPffr9WPl9Y
2,the city of edinburgh has decorated herself wi...,39.257000,43.437,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y
3,so today we will explore different places in e...,43.796000,50.094,so today we will explore different places in e...,tPffr9WPl9Y
4,so lets go with me,50.504000,51.928,so lets go with me,tPffr9WPl9Y
...,...,...,...,...,...
4345684,it is reported that as a result of artillery f...,167.800000,173.610,it is reported that as a result of artillery f...,78VHQeebbWM
4345685,region the russian troops lost about,176.202537,179.400,the city of starobilsk luhansk region the russ...,78VHQeebbWM
4345686,general staff of the armed forces of ukraine o...,183.030000,190.590,general staff of the armed forces of ukraine o...,78VHQeebbWM
4345687,you can express your thoughts in the comments ...,193.159000,196.769,you can express your thoughts in the comments ...,78VHQeebbWM


In [26]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: (x-shift))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: (x+shift))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,jingle bell jingle bell jingle all the way,29.116000,33.905,jingle bell jingle bell jingle all the way,tPffr9WPl9Y
1,what else do you wish for in this festive season,36.850000,39.300,what else do you wish for in this festive season,tPffr9WPl9Y
2,the city of edinburgh has decorated herself wi...,38.957000,43.737,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y
3,so today we will explore different places in e...,43.496000,50.394,so today we will explore different places in e...,tPffr9WPl9Y
4,so lets go with me,50.204000,52.228,so lets go with me,tPffr9WPl9Y
...,...,...,...,...,...
4345684,it is reported that as a result of artillery f...,167.500000,173.910,it is reported that as a result of artillery f...,78VHQeebbWM
4345685,region the russian troops lost about,175.902537,179.700,the city of starobilsk luhansk region the russ...,78VHQeebbWM
4345686,general staff of the armed forces of ukraine o...,182.730000,190.890,general staff of the armed forces of ukraine o...,78VHQeebbWM
4345687,you can express your thoughts in the comments ...,192.859000,197.069,you can express your thoughts in the comments ...,78VHQeebbWM


In [27]:
df_word_group_time_loc_in_sent_result["start_time"] = df_word_group_time_loc_in_sent_result["start_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result["end_time"] = df_word_group_time_loc_in_sent_result["end_time"].apply(lambda x: round(x))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,jingle bell jingle bell jingle all the way,29,34,jingle bell jingle bell jingle all the way,tPffr9WPl9Y
1,what else do you wish for in this festive season,37,39,what else do you wish for in this festive season,tPffr9WPl9Y
2,the city of edinburgh has decorated herself wi...,39,44,the city of edinburgh has decorated herself wi...,tPffr9WPl9Y
3,so today we will explore different places in e...,43,50,so today we will explore different places in e...,tPffr9WPl9Y
4,so lets go with me,50,52,so lets go with me,tPffr9WPl9Y
...,...,...,...,...,...
4345684,it is reported that as a result of artillery f...,168,174,it is reported that as a result of artillery f...,78VHQeebbWM
4345685,region the russian troops lost about,176,180,the city of starobilsk luhansk region the russ...,78VHQeebbWM
4345686,general staff of the armed forces of ukraine o...,183,191,general staff of the armed forces of ukraine o...,78VHQeebbWM
4345687,you can express your thoughts in the comments ...,193,197,you can express your thoughts in the comments ...,78VHQeebbWM


In [28]:
df_word_group_time_loc_in_sent_result.sort_values("search_string",key=lambda x:x.str.len(), inplace=True, ascending=False)
df_word_group_time_loc_in_sent_result.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id
0,and speed in everything you have done in your ...,871,872,i have seen a great of flexibility and speed i...,mBVqmanRkEE
1,the public health messages it s right that the...,332,338,well we know from what the prime minister has ...,VWd03ovFlas
2,it i think another is that we vote at times we...,871,872,idea versus jocelyns idea we dont talk about l...,mBVqmanRkEE
3,xu awesome um i think im gon na start with my ...,1009,1039,kailyn xu awesome um i think im gon na start w...,iJkfTPN8MS0
4,ultimately boris johnson needs to be able to c...,332,338,ultimately boris johnson needs to be able to c...,VWd03ovFlas
...,...,...,...,...,...
4345684,e i e i o,206,209,e i e i o,DsGvkftHHzE
4345685,e i e i o,549,552,e i e i o,mEbuhmbeT80
4345686,e i e i o,559,562,e i e i o,mEbuhmbeT80
4345687,e i e i o,569,572,e i e i o,mEbuhmbeT80


In [29]:
df_word_group_time_loc_in_sent_result["unique_word_ratio"] = df_word_group_time_loc_in_sent_result["search_string"].apply(lambda text: repetation_ratio(text))
df_word_group_time_loc_in_sent_result

,search_string,start_time,end_time,sentence,video_id,unique_word_ratio
0,and speed in everything you have done in your ...,871,872,i have seen a great of flexibility and speed i...,mBVqmanRkEE,61
1,the public health messages it s right that the...,332,338,well we know from what the prime minister has ...,VWd03ovFlas,68
2,it i think another is that we vote at times we...,871,872,idea versus jocelyns idea we dont talk about l...,mBVqmanRkEE,60
3,xu awesome um i think im gon na start with my ...,1009,1039,kailyn xu awesome um i think im gon na start w...,iJkfTPN8MS0,63
4,ultimately boris johnson needs to be able to c...,332,338,ultimately boris johnson needs to be able to c...,VWd03ovFlas,64
...,...,...,...,...,...,...
4345684,e i e i o,206,209,e i e i o,DsGvkftHHzE,60
4345685,e i e i o,549,552,e i e i o,mEbuhmbeT80,60
4345686,e i e i o,559,562,e i e i o,mEbuhmbeT80,60
4345687,e i e i o,569,572,e i e i o,mEbuhmbeT80,60


In [30]:
df_word_group_time_loc_in_sent_result_repeat_ratio = df_word_group_time_loc_in_sent_result[df_word_group_time_loc_in_sent_result["unique_word_ratio"] > unique_word_ratio]
df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-30-d6186b820154>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop_duplicates(subset=["search_string","start_time","end_time"], inplace=True)
<ipython-input-30-d6186b820154>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio.drop("unique_word_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,that this is not just getting any old attorney...,5513,5527,katie kersh that this is not just getting any ...,gasLdUX61TU
1,thats something you dont have to worry about a...,2199,2213,lauren opgenorth thats something you dont have...,HWTd4jxCixk
2,so prepare relevant questions and also on the ...,1444,1462,isadora ocampo so prepare relevant questions a...,qgBqBnEJFgs
3,james moore depending on the graduation someti...,3463,3484,james moore depending on the graduation someti...,9nBekn1eQtg
4,this type of research yeah i think thats still...,3706,3724,this type of research yeah i think thats still...,Anatl2PLRag
...,...,...,...,...,...
3294696,c h a o s,408,411,c h a o s,S38wovcLwjQ
3294697,b i n g o,24,27,b i n g o,KPPwNxu7O_g
3294698,b i n g o,27,30,b i n g o,KPPwNxu7O_g
3294699,b i n g o,30,33,b i n g o,KPPwNxu7O_g


In [31]:
df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)
df_word_group_time_loc_in_sent_result_repeat_ratio

<ipython-input-31-e137816795b3>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] = df_word_group_time_loc_in_sent_result_repeat_ratio.apply(lambda x: int((len(str(x['search_string']))/len(str(x['sentence'])))*100) ,axis=1)


,search_string,start_time,end_time,sentence,video_id,include_string_ratio
0,that this is not just getting any old attorney...,5513,5527,katie kersh that this is not just getting any ...,gasLdUX61TU,95
1,thats something you dont have to worry about a...,2199,2213,lauren opgenorth thats something you dont have...,HWTd4jxCixk,93
2,so prepare relevant questions and also on the ...,1444,1462,isadora ocampo so prepare relevant questions a...,qgBqBnEJFgs,94
3,james moore depending on the graduation someti...,3463,3484,james moore depending on the graduation someti...,9nBekn1eQtg,95
4,this type of research yeah i think thats still...,3706,3724,this type of research yeah i think thats still...,Anatl2PLRag,100
...,...,...,...,...,...,...
3294696,c h a o s,408,411,c h a o s,S38wovcLwjQ,100
3294697,b i n g o,24,27,b i n g o,KPPwNxu7O_g,100
3294698,b i n g o,27,30,b i n g o,KPPwNxu7O_g,100
3294699,b i n g o,30,33,b i n g o,KPPwNxu7O_g,100


In [32]:
df_word_group_time_loc_include_string_ratio = df_word_group_time_loc_in_sent_result_repeat_ratio[df_word_group_time_loc_in_sent_result_repeat_ratio["include_string_ratio"] > include_wordgroup_ratio]
df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)
df_word_group_time_loc_include_string_ratio.reset_index(drop=True, inplace=True)
df_word_group_time_loc_include_string_ratio

<ipython-input-32-ea709f90da2b>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_word_group_time_loc_include_string_ratio.drop("include_string_ratio", axis=1, inplace=True)


,search_string,start_time,end_time,sentence,video_id
0,that this is not just getting any old attorney...,5513,5527,katie kersh that this is not just getting any ...,gasLdUX61TU
1,thats something you dont have to worry about a...,2199,2213,lauren opgenorth thats something you dont have...,HWTd4jxCixk
2,so prepare relevant questions and also on the ...,1444,1462,isadora ocampo so prepare relevant questions a...,qgBqBnEJFgs
3,james moore depending on the graduation someti...,3463,3484,james moore depending on the graduation someti...,9nBekn1eQtg
4,this type of research yeah i think thats still...,3706,3724,this type of research yeah i think thats still...,Anatl2PLRag
...,...,...,...,...,...
2305482,c h a o s,408,411,c h a o s,S38wovcLwjQ
2305483,b i n g o,24,27,b i n g o,KPPwNxu7O_g
2305484,b i n g o,27,30,b i n g o,KPPwNxu7O_g
2305485,b i n g o,30,33,b i n g o,KPPwNxu7O_g


In [33]:
df_youtube_word_usage = word_usage_result(word_list, df_word_group_time_loc_include_string_ratio, "search_string", "video_id", word_usage_min, word_usage_max)
df_youtube_word_usage.drop_duplicates(inplace=True)
df_youtube_word_usage

,search_string,video_id
0,since grandma went amp picked pea leaves,zzgWjSzQ9yU
1,bridget jones fianc wife mother,zyOCtVxkRy4
2,why does she choose television specifically,zyOCtVxkRy4
3,support grid rather than specifically targeting,zyHhANpL2xk
4,our special musical performances uh for today ...,zyEMeStyuB8
...,...,...
6823,then everything probably flies off,01J8qKjcp0M
6824,hair okay skin cells yep,01J8qKjcp0M
6825,quarter million cubic foot sucked every minute,01J8qKjcp0M
6826,makeup carbon dirt et cetera,01J8qKjcp0M


In [34]:
df_youtube_word_usage_merge = pd.merge(df_word_group_time_loc_include_string_ratio,df_youtube_word_usage,how="inner",on=["search_string","video_id"])
df_youtube_word_usage_merge.drop_duplicates(subset=["search_string","video_id"],inplace=True)
df_youtube_word_usage_merge.reset_index(drop=True, inplace=True)
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id
0,that this is not just getting any old attorney...,5513,5527,katie kersh that this is not just getting any ...,gasLdUX61TU
1,thats something you dont have to worry about a...,2199,2213,lauren opgenorth thats something you dont have...,HWTd4jxCixk
2,so prepare relevant questions and also on the ...,1444,1462,isadora ocampo so prepare relevant questions a...,qgBqBnEJFgs
3,james moore depending on the graduation someti...,3463,3484,james moore depending on the graduation someti...,9nBekn1eQtg
4,this type of research yeah i think thats still...,3706,3724,this type of research yeah i think thats still...,Anatl2PLRag
...,...,...,...,...,...
6754,q r s t u v,19,24,q r s t u v,CgoqMK0P9_U
6755,l m n o p,109,110,l m n o p,EagLlPDC3Tw
6756,l m n o p,650,655,l m n o p,6aYMOPerAz8
6757,d e f g h,324,327,d e f g h,wKC7h0uBFV4


In [35]:
df_youtube_word_usage_merge["video_url"] = "https://www.youtube.com/watch?v="+df_youtube_word_usage_merge['video_id'].map(str)+"&t="+df_youtube_word_usage_merge['start_time'].map(str)+"s"
df_youtube_word_usage_merge

,search_string,start_time,end_time,sentence,video_id,video_url
0,that this is not just getting any old attorney...,5513,5527,katie kersh that this is not just getting any ...,gasLdUX61TU,https://www.youtube.com/watch?v=gasLdUX61TU&t=...
1,thats something you dont have to worry about a...,2199,2213,lauren opgenorth thats something you dont have...,HWTd4jxCixk,https://www.youtube.com/watch?v=HWTd4jxCixk&t=...
2,so prepare relevant questions and also on the ...,1444,1462,isadora ocampo so prepare relevant questions a...,qgBqBnEJFgs,https://www.youtube.com/watch?v=qgBqBnEJFgs&t=...
3,james moore depending on the graduation someti...,3463,3484,james moore depending on the graduation someti...,9nBekn1eQtg,https://www.youtube.com/watch?v=9nBekn1eQtg&t=...
4,this type of research yeah i think thats still...,3706,3724,this type of research yeah i think thats still...,Anatl2PLRag,https://www.youtube.com/watch?v=Anatl2PLRag&t=...
...,...,...,...,...,...,...
6754,q r s t u v,19,24,q r s t u v,CgoqMK0P9_U,https://www.youtube.com/watch?v=CgoqMK0P9_U&t=19s
6755,l m n o p,109,110,l m n o p,EagLlPDC3Tw,https://www.youtube.com/watch?v=EagLlPDC3Tw&t=...
6756,l m n o p,650,655,l m n o p,6aYMOPerAz8,https://www.youtube.com/watch?v=6aYMOPerAz8&t=...
6757,d e f g h,324,327,d e f g h,wKC7h0uBFV4,https://www.youtube.com/watch?v=wKC7h0uBFV4&t=...


In [36]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(2000)
df_youtube_word_usage_merge.to_csv(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.csv", index=False)

In [37]:
df_youtube_word_usage_merge = df_youtube_word_usage_merge.head(700)
df_youtube_word_usage_merge.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

In [38]:
#df_word_group_time_loc_in_sent_result = df_word_group_time_loc_in_sent_result.head(1000000)
#df_word_group_time_loc_in_sent_result.to_excel(f"{lang_folder.capitalize()}_Word_Group_Max_In_Youtube_Sentence_{word_end}_Word{file_ext}.xlsx", index=False)

#### Copy Move And Delete

In [39]:
output_file = glob.glob(f"{lang_folder.capitalize()}_Word_Group*{file_ext}.*")
output_file

['English_Word_Group_Max_In_Youtube_Sentence_20000_Word6.csv',
 'English_Word_Group_Max_In_Youtube_Sentence_20000_Word6.xlsx']

In [40]:
for k in output_file:
    source = k # source directory
    destination = path
    shutil.copy2(source, destination)

In [41]:
for i in output_file:
    try:
        os.remove(i)
    except:
        pass